In [1]:
import os

In [4]:
%pwd

'C:\\prabhakar\\end_to_end\\text_summerizer'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path
    

In [11]:
from Text_Summerizer.constants import *
from Text_Summerizer.utils.common import read_yaml,create_directiories

In [33]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directiories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion

        create_directiories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir)
        return data_ingestion_config

In [34]:
import os
import urllib.request as request
import zipfile
from Text_Summerizer.logging import logger
from Text_Summerizer.utils.common import get_size

In [37]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """Extracts the zip file into the data directory."""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [38]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-22 17:12:26,395 | INFO     | TextSummarizerLogger | read_yaml           :29  ] yaml file:config\config.yaml loadded successfully
[2024-01-22 17:12:26,396 | INFO     | TextSummarizerLogger | read_yaml           :29  ] yaml file:params.yaml loadded successfully
[2024-01-22 17:12:26,397 | INFO     | TextSummarizerLogger | create_directiories :49  ] created directory at: artifacts
[2024-01-22 17:12:26,398 | INFO     | TextSummarizerLogger | create_directiories :49  ] created directory at: artifacts/data_ingestion
[2024-01-22 17:12:34,102 | INFO     | TextSummarizerLogger | download_file       :11  ] artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: no